# Audio Leak Detection Model Training

This notebook trains a model to classify audio files as:
- Leak-Metal
- Leak-NonMetal
- NoLeak-Metal
- NoLeak-NonMetal


## 1. Setup Environment

First, install required packages and mount Google Drive (if your data is stored there).


In [ ]:
# Install required packages
!pip install -q -r requirements.txt


In [ ]:
# Mount Google Drive (if your project is in Drive)
# Uncomment the following lines if needed:
# from google.colab import drive
# drive.mount('/content/drive')

# If uploading files directly to Colab, use:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")


## 2. Data Loading and Preprocessing

Define paths and load audio files. Adjust the base path according to your setup.


In [ ]:
# Define base path
# Adjust this path based on where your data is located
# If in Drive: BASE_PATH = '/content/drive/MyDrive/ECO5000 - Modelos IA'
# If uploaded: BASE_PATH = '/content'
# If in repository: BASE_PATH = '/content'

BASE_PATH = '/content'  # Change this as needed
AUDIO_DIR = os.path.join(BASE_PATH, 'Audios para Treinamento')

# Verify directory exists
if os.path.exists(AUDIO_DIR):
    print(f"Audio directory found: {AUDIO_DIR}")
    print("\nClass directories:")
    for class_dir in os.listdir(AUDIO_DIR):
        class_path = os.path.join(AUDIO_DIR, class_dir)
        if os.path.isdir(class_path):
            num_files = len([f for f in os.listdir(class_path) if f.endswith('.wav')])
            print(f"  {class_dir}: {num_files} files")
else:
    print(f"Warning: Audio directory not found at {AUDIO_DIR}")
    print("Please adjust BASE_PATH or upload your data.")


In [ ]:
def extract_features(file_path, n_mfcc=13, n_mels=128, n_fft=2048, hop_length=512):
    """
    Extract audio features from a WAV file.
    Returns MFCCs, Mel spectrogram, and chroma features.
    """
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=None)
        
        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        
        # Extract Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, 
                                                   n_fft=n_fft, hop_length=hop_length)
        mel_spec_mean = np.mean(mel_spec.T, axis=0)
        
        # Extract Chroma features
        chroma = librosa.feature.chroma(y=y, sr=sr)
        chroma_mean = np.mean(chroma.T, axis=0)
        
        # Combine features
        features = np.concatenate([mfccs_mean, mel_spec_mean, chroma_mean])
        
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

print("Feature extraction function defined.")


In [ ]:
def load_dataset(audio_dir):
    """
    Load all audio files and extract features.
    """
    features = []
    labels = []
    file_paths = []
    
    class_dirs = [d for d in os.listdir(audio_dir) if os.path.isdir(os.path.join(audio_dir, d))]
    
    for class_name in class_dirs:
        class_path = os.path.join(audio_dir, class_name)
        audio_files = [f for f in os.listdir(class_path) if f.endswith('.wav')]
        
        print(f"\nProcessing {class_name}... ({len(audio_files)} files)")
        
        for audio_file in tqdm(audio_files, desc=f"  {class_name}"):
            file_path = os.path.join(class_path, audio_file)
            feature = extract_features(file_path)
            
            if feature is not None:
                features.append(feature)
                labels.append(class_name)
                file_paths.append(file_path)
    
    return np.array(features), np.array(labels), file_paths

# Load the dataset
print("Loading dataset...")
X, y, paths = load_dataset(AUDIO_DIR)

print(f"\nDataset loaded:")
print(f"  Features shape: {X.shape}")
print(f"  Labels shape: {y.shape}")
print(f"  Classes: {np.unique(y)}")
print(f"  Class distribution:")
for class_name, count in zip(*np.unique(y, return_counts=True)):
    print(f"    {class_name}: {count}")


## 3. Data Preprocessing and Splitting


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Label mapping:")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"  {i}: {class_name}")

num_classes = len(label_encoder.classes_)
print(f"\nNumber of classes: {num_classes}")


In [ ]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Further split training set into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Dataset split:")
print(f"  Training: {X_train.shape[0]} samples")
print(f"  Validation: {X_val.shape[0]} samples")
print(f"  Test: {X_test.shape[0]} samples")
print(f"  Feature dimension: {X_train.shape[1]}")


In [ ]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Features normalized.")


## 4. Build and Train Model


In [ ]:
def create_model(input_dim, num_classes):
    """
    Create a neural network model for audio classification.
    """
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create model
model = create_model(X_train_scaled.shape[1], num_classes)
model.summary()


In [ ]:
# Define callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001
    )
]

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


## 5. Evaluate Model


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Predictions
y_pred = model.predict(X_test_scaled, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(history.history['accuracy'], label='Train Accuracy')
ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(history.history['loss'], label='Train Loss')
ax2.plot(history.history['val_loss'], label='Val Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()


## 6. Save Model

Save the trained model for later use.


In [ ]:
# Save model
model.save('audio_leak_classifier.h5')
print("Model saved as 'audio_leak_classifier.h5'")
